# Playing with the gensim coherence measures

## Introduction

Testing out difference 'coherence' pipeline combinations.
Topic models: LDA, NNMF, LSA, PCA + varimax

In [76]:
%load_ext autoreload
%autoreload 2
import logging
import warnings
import numpy as np
import pandas as pd
import io
from process_topics import show_topic_words, run_all, coherence_widget, NewCoherence
from gensim.models import CoherenceModel, LdaModel, HdpModel, nmf, LdaMulticore
from gensim.corpora import Dictionary, csvcorpus
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import *
from time import time
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual, fixed, interactive_output
import ipywidgets as widgets
import qgrid
import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set up corpus

Set up test corpus, should be tokenized, sotpwords removed

In [3]:
%%time
# my_csv = pd.read_csv('..\..\wiki_movie_plots_deduped.csv')
# plots = my_csv['Plot']
elections = io.open('Election2008Paragraphes.txt',encoding = "ISO-8859-1")
e  =elections.readlines()

CUSTOM_FILTERS = [lambda x: x.lower(),  strip_punctuation, strip_multiple_whitespaces, strip_numeric,remove_stopwords, strip_short]

texts = [preprocess_string(line, filters=CUSTOM_FILTERS) for line in e]


dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
data = [texts, dictionary, corpus]


Wall time: 1.29 s


In [4]:
#changes made to FARotate in the code will require kernel restart if reloading this cell!
m = coherence_widget(data)
display(m)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFJhZGlvQnV0dG9ucyhkZXNjcmlwdGlvbj11J0Nob29zZSBNb2RlbCcsIGxheW91dD1MYXlvdXQod2lkdGg9dScyNTBweCcpLCBvcHRpb25zPSgnTk1GJyzigKY=


In [5]:
topics, coherences, both= m.result

In [6]:
topics

[{'Topic': u'years troops today ago question safer percent gas bring stations'},
 {'Topic': u'oil america china new radical transportation million regimes security energy'},
 {'Topic': u'plan opposed irish general need biden year security fund americans'},
 {'Topic': u'iraq sunnis oil interests federalism revenues constitution shiites kurds work'},
 {'Topic': u'government foreign trust central oil sunnis policy capacity relations ago'},
 {'Topic': u'iraq plan president administration troops leaving necessary strategy leave home'},
 {'Topic': u'way iraq political best strategy america states start help military'},
 {'Topic': u'war civil president middle end wrong afghanistan cars regional send'},
 {'Topic': u'iraq committee political policy want options public dynamics main national'},
 {'Topic': u'american billion year irish fund biden mother new senator people'}]

In [159]:
from process_topics import coherence_scores
from similarity import *


In [182]:
style = {'width': '1000px', 'height': '30px'}

w=IntSlider(10,0,1000, layout=style)
w2=IntSlider(10,0,1000, layout=style) 
co=IntSlider(10,0,1000, layout=style)
ndocs=IntSlider(1000,0,10000, layout=style)
exp = IntSlider(1000,0,10000, layout=style)
l = ipywidgets.link(co, exp)
ui = widgets.VBox([w, w2, co, ndocs])

def similarities(w,w2,co,ndocs, held=[]):
    scores = calculate_sims(w,w2,co,ndocs,'all')
    #held.append(w)
    display(pd.DataFrame(scores))
    return scores
style = {'width': '1000px'}
e =interactive_output(similarities,
         {'w':w,'w2':w2,'co':co,'ndocs':ndocs})
e.layout.height= '500px'
display(ui,e)


NameError: name 'ipywidgets' is not defined

In [149]:
from ipywidgets import *
from IPython.display import display, HTML
w = IntSlider()
display(w)

out = Output()
display(out)

@out.capture()
def callback(change):
    display(change)

w.observe(callback, names=['value'])


IntSlider(value=0)

Output()

AttributeError: 'IntSlider' object has no attribute 'onclick'

In [127]:


from itertools import combinations
l = range(10)
cl =list(combinations(l,2))
print cl
print(len(cl))
e = None


[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 6), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9), (7, 8), (7, 9), (8, 9)]
45


In [121]:

print np.average(combo2,weights=weights)
topics= [['years','troops','capital'],['years','troops']]

from segmentation import segment_with_weights
t,w= segment_with_weights(topics)
w=[]
print zip(t,w)

17.8666666667
[]


In [44]:
from sklearn.feature_extraction.text import CountVectorizer
docs = ['this this this book',
        'this cat good',
        'cat good shit']
count_model = CountVectorizer(ngram_range=(1,1)) # default unigram model
X = count_model.fit_transform(docs)
# X[X > 0] = 1 # run this line if you don't want extra within-text cooccurence (see below)
Xc = (X.T * X) # this is co-occurrence matrix in sparse csr format
Xc.setdiag(0) # sometimes you want to fill same word cooccurence to 0
print(Xc.todense()) # print out matrix in dense format
print Xc

[[0 0 0 0 3]
 [0 0 2 1 1]
 [0 2 0 1 1]
 [0 1 1 0 0]
 [3 1 1 0 0]]
  (4, 0)	3
  (0, 0)	0
  (3, 1)	1
  (4, 1)	1
  (1, 1)	0
  (2, 1)	2
  (3, 2)	1
  (4, 2)	1
  (1, 2)	2
  (2, 2)	0
  (1, 3)	1
  (2, 3)	1
  (3, 3)	0
  (1, 4)	1
  (2, 4)	1
  (4, 4)	0
  (0, 4)	3


In [163]:
print int(3254)/ 234345435.6

1.38854848684e-05
